# Homework 10: Group 1
#### Random Forest (Washington DC Bikeshare data)

David Jung, Ryan Voges, Emily Blake, Spencer Powell, Abraham Alhomadi

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  #if you want to use seaborn themes with matplotlib functions
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')


In [10]:
rand_state = 1000

In [2]:
df=pd.read_csv('bikeshare.csv')

In [3]:
df.head(10)

,season,yr,mnth,hr,holiday,weekday,notbizday,weathersit,temp,hum,windspeed,dteday,cnt
0,1,0,1,0,0,6,1,1,-1.334609,0.947345,-1.553844,2011-01-01,16
1,1,0,1,1,0,6,1,1,-1.438475,0.895513,-1.553844,2011-01-01,40
2,1,0,1,2,0,6,1,1,-1.438475,0.895513,-1.553844,2011-01-01,32
3,1,0,1,3,0,6,1,1,-1.334609,0.636351,-1.553844,2011-01-01,13
4,1,0,1,4,0,6,1,1,-1.334609,0.636351,-1.553844,2011-01-01,1
5,1,0,1,5,0,6,1,2,-1.334609,0.636351,-0.821460,2011-01-01,1
6,1,0,1,6,0,6,1,1,-1.438475,0.895513,-1.553844,2011-01-01,2
7,1,0,1,7,0,6,1,1,-1.542341,1.206507,-1.553844,2011-01-01,3
8,1,0,1,8,0,6,1,1,-1.334609,0.636351,-1.553844,2011-01-01,8
9,1,0,1,9,0,6,1,1,-0.919146,0.688184,-1.553844,2011-01-01,14


In [4]:
for col in df.columns:
    print(col, df[col].nunique())

season 4
yr 2
mnth 12
hr 24
holiday 2
weekday 7
notbizday 2
weathersit 4
temp 50
hum 89
windspeed 30
dteday 731
cnt 869


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  int64  
 1   yr          17379 non-null  int64  
 2   mnth        17379 non-null  int64  
 3   hr          17379 non-null  int64  
 4   holiday     17379 non-null  int64  
 5   weekday     17379 non-null  int64  
 6   notbizday   17379 non-null  int64  
 7   weathersit  17379 non-null  int64  
 8   temp        17379 non-null  float64
 9   hum         17379 non-null  float64
 10  windspeed   17379 non-null  float64
 11  dteday      17379 non-null  object 
 12  cnt         17379 non-null  int64  
dtypes: float64(3), int64(9), object(1)
memory usage: 1.7+ MB


In [6]:
dummies = ["mnth", "hr", "weathersit"]

In [7]:
df = pd.get_dummies(df, drop_first = True, columns = dummies)

**Question 1: RF Regression**

In this exercise I want you to apply RF regression model to the bikeshare data set which is available on the GitHub folder for HW10. From HW-7 KNN (Washington DC Bikeshare data)  we are already familiar with this data set so there is no need to do an EDA. The data are 17379 observations of hourly counts from 2011 to 2012 for bike rides (rentals) from the Capital Bikeshare system in Washington DC. It was originally compiled by Fanaee and Gama in ‘Event labeling combining ensemble detectors and background knowledge’ (2013).
Import the bikeshare.csv as a data frame and name it df.

1. Define your feature space and target variables. Split the data into test (30%) and train set (70%)

In [11]:
# Defining our target variable and the features and splitting the dataset into train and test:
y = df['cnt']
X = df.drop('cnt', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.3, random_state=rand_state)

In [12]:
X_train.head()

,season,yr,holiday,weekday,notbizday,temp,hum,windspeed,dteday,mnth_2,...,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,weathersit_2,weathersit_3,weathersit_4
2243,2,0,0,5,0,-0.815280,1.258339,-0.211685,2011-04-08,0,...,0,0,0,0,0,0,0,0,1,0
17261,1,1,0,4,0,-1.334609,1.258339,2.106440,2012-12-27,0,...,0,0,0,0,0,0,0,1,0,0
435,1,0,0,4,0,-1.230743,-0.348463,-0.211685,2011-01-20,0,...,0,0,0,0,0,0,0,0,0,0
785,1,0,0,6,1,-1.438475,1.569333,-0.456086,2011-02-05,1,...,0,0,0,0,0,0,0,0,1,0
6360,4,0,0,2,0,0.950439,0.843681,-0.699669,2011-09-27,0,...,0,0,0,0,0,0,0,1,0,0


2. From sklearn.ensemble import the relevant function for RF regression. Do the followings: 
    1. Train the model with the default features. However use random_state=1000.
    2. Make predictions on the test set and save them as y_hat 
    3. Construct a data frame named df_predictions with 2 columns. y_test, and y_hat  from previous part 
    4. Visualize actual vs predicted counts in the test set using an scatterplot. Are you visually satisfied with the regression model? 
    5. Report the R-squared and RMSE_test for the RFF regression model. 

3. Tuning hyperparameters: I specifically want you to use the following param_grid. my_param_grid = {'n_estimators':[100,200], 'max_features':['log2', 'auto'], 'max_depth':[10,None]} . Where you able to improve the model performance? Is random forest using a small tree or bushy tree? How do yo know?

4. Cross validation and OOB observations: estimate the R2 of the test set using both CV and OOB methods and compare the results.

5. Feature importance: Plot the feature importance graph and compare your top 5 important features with the top 5 most significant variables derived from a linear regression model.  Can you name a variable which was important according to RF but not significant according to linear regression model? What's going on here?

**Question 2: RF Classification**

The managers of Capital Bikeshare have found that the system works smoothly until more than 500 bikes are rented in any one hour. At that point, it becomes necessary to insert extra bikes into the system and move them across stations to balance loads. Do the followings

1. Define a binary target variable overload. Overload=1 if cnt>500 and 0 otherwise. What are the proportions of overload vs non-overload in your data set? Is the target variable balanced or imbalanced?

2. Along with the target variable, define your feature space (X) and split the data into test (30%) and train set (70%

3. From sklearn.ensemble import the relevant function for RF classification. Do the followings: (25 points)
    1. Train the RF classification model using its default parameters. However use random_state=1000. 
    2. Generate the predicted probabilities and predicted classifications and save them as y_hat_probs, y_hat respectively.
    3. Plot the histogram of y_hat_probs? Explain what you see? Is there a probability threshold at which the model always predict negative or positive? 
    4. Use the built-in classification report function from sklearn. Report the Accuracy, precision, recall and f1 score along with the confusion matrix. Interpret all of these statistics. Do you trust the accuracy of the model? why? 

4. Tuning hyperparameters: I specifically want you to use the following param_grid. my_param_grid = {'n_estimators':[100,200], 'criterion':['gini','entropy'],  'max_features':['log2', 'auto'], 'max_depth':[10,None]} . Where you able to improve the model performance? Is random forest using a small tree or bushy tree? How do yo know?

5. Cross validation and OOB observations: estimate the accuracy of the test set using both CV and OOB methods and compare the results. 

6. Dealing with imbalanced data: Use the balanced version of RF classifier and save the predictions as y_hat_balanced. Report the precision, recall and f1 score for the balanced RF versus what you found in part 3.4. What happened to recall? does that make sense?

7. Plot the ROC curve for the balanced RF and report the AUC. Can you trust this number now? why?

8. Feature importance: Plot the feature importance graph and report the top 5 important features. Is your finding consistent with common sense?